In [ ]:
# নিচের ব্লক টি রান করতে হবে ডিপেনড্যান্সিগুলো ইন্সটল করার জন্য 

In [1]:
# Run this block to install dependencies [Remember to make the statement true]
if 0 == 1:
    !pip3 install pandas
    !pip3 install tqdm
    !pip3 install scikit-learn
    !pip3 install gensim
    !pip3 install spacy
    !python3 -m spacy download en
    !pip3 install numpy

In [2]:
import pandas as pd
import spacy
import pickle
from tqdm import tqdm
from gensim.corpora import Dictionary
from sklearn.utils import shuffle
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

C:\Users\Rezwana\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [14]:

nlp = spacy.load('bn')
doc = nlp(u'আমার োনার বাংলা')
doc



    Only loading the 'bn' tokenizer.



আমার োনার বাংলা

In [15]:
# load model with shortcut link "en"
nlp = spacy.load('en')


DATA_LIMIT = 1000

# df stands for data frame
df = pd.read_csv('./imdb_master.csv', encoding='latin1')

df_neg = df[df['label'] == 'neg']
df_pos = df[df['label'] == 'pos']

df = pd.concat((df_pos[:DATA_LIMIT], df_neg[:DATA_LIMIT]))

df_test=pd.concat((df_pos[1000:1500], df_neg[1000:1500]))

#removing stop word, converting text to lower case, removing digits

def process_text(input_string, return_string=False, stem=False):
    text = nlp(u'' + input_string)
    if stem == True:
        text = [tok.lemma_ for tok in text if (tok.is_alpha and not tok.is_stop)]
    else:
        text = [tok.lower_ for tok in text if (tok.is_alpha and not tok.is_stop)]
    if return_string == True:
        return " ".join(text)
    return text

In [22]:
# Make this statement true to run from scratch [It takes time to process the text]

#tqdm is for the progress bar
#df.shape[0] is used for number of row count
#df.shape[1] is used for number of column count
#wordlist contains all the words in the dataframe except stop words and digits, all lowercase
#wb for writing in binary

if 1 == 0:
    wordlist = []
    
    for i in tqdm(range(df.shape[0])): 
        
        wordlist.append(process_text(df['review'].iloc[i]))
        
    with open('vocabulary.txt', 'wb') as vocabulary:
        pickle.dump(wordlist, vocabulary) #writing wordlist to vocabulary
    vocabulary.close()

In [23]:
# Load vocabulary
wordlist = []
with open('vocabulary.txt', 'rb') as vocabulary:
    wordlist = pickle.load(vocabulary) # wordlist <-- vocabulary

In [43]:
# Keeping track of frequency of a single token
frequency = defaultdict(int) #defaultdict(int, {})
for text in wordlist:
    for token in text:
        frequency[token] += 1 #counting frequency of each token
        
# Apply Threshold to limit the vocabulary size, discarding the tokens which appeard number of times below the threshold limit 
FREQ_THRESHOLD = 35 # set a threshold to get the words who are frequency more than 35

thresholded_wordlist =  [[token for token in text if frequency[token] > FREQ_THRESHOLD]
          for text in wordlist] # thresholded_wordlist contains the words who have frq more than 35, contains duplicate value

# Create Dictionary based on the word list
dictionary = Dictionary(thresholded_wordlist) # unique words are in dictionary

# Number of tokens
print("Number of Tokens - {}".format(len(dictionary)))

Number of Tokens - 1088


In [44]:
# now dumping the words from the dataframe in lowercase format
with open('dump_skipgram.txt','w', encoding='utf-8') as f:
    for i in range(2000):
        f.write(df['review'].iloc[i].lower())
    f.close()
    

In [47]:
#storing the words from data frame to dump variable 
#tqdm for the progress bar
dump=""
for i in tqdm(range(2000)):
         dump+=(df['review'].iloc[i].lower())
    

100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 4913.05it/s]


In [50]:
# taking the whole dump sentence on spacy_obj
spacy_obj=nlp(u''+dump) 

In [51]:
len(spacy_obj)

525604

In [53]:
#one hot representation 1088*1088 dimention
#one_hot=np.eye(1088)

In [52]:
#window size, m=2 
m=2

In [54]:
# inversing the dictionary; 
#word2idx contains word to index
#will return index for a given word id it exists

word2idx = { dictionary[i]:i for i, word in enumerate(dictionary) }

In [55]:
word2idx['able']

0

In [56]:
word2idx['i']

11

In [57]:
# this function will return the index if found on word2idx 
#else return -1

def getIndex(word):
    try:
        idx=word2idx[word]
    except KeyError:
        idx=-1
    return idx


In [58]:
#finding the index of left and right words of the center word according to the window size
def input_generation(word):
    
    inp1=np.zeros(4)
    index=getIndex(str(word.nbor(-2)))   
    inp1[0]=index
    index=getIndex(str(word.nbor(-1)))   
    inp1[1]=index
    index=getIndex(str(word.nbor(1)))   
    inp1[2]=index
    index=getIndex(str(word.nbor(2)))   
    inp1[3]=index
    return inp1      


In [59]:
len_s=len(spacy_obj)

inp=np.zeros((len_s,4)) # for each word in the spacy_obj spaces are created according to the windowsize
target=np.zeros(len_s) # target has 1*len_s dimention
for i in tqdm(range(len_s-2)):
    
    token=spacy_obj[i]
#     print(token)
    token1=str(token) #need to convert to string to send to the getIndex function
    index=getIndex(token1)
    target[i]=index #storing the index of the token in target 
#     print(target[i])
    inp[i,:]=input_generation(spacy_obj[i]) # creating the input window for the word in the index i
      


100%|███████████████████████████████████████████████████████████████████████| 525602/525602 [00:07<00:00, 71938.14it/s]


In [61]:
inp.shape
inp=inp[np.where(target!=-1)[0]] # storing the indexs whose target value found in the dictionary 

In [62]:
target_positive=target[np.where(target!=-1)[0]] # removing the indexs with -1

In [63]:
target_positive[0]

11.0

In [64]:
target_positive.shape

(291908,)

In [65]:
# keras help: https://keras.io/getting-started/sequential-model-guide/
# keras is taken to calculate mean value according to column
# to implement neural network
#The Sequential model is a linear stack of layers.

# Some 2D layers, such as Dense, support the specification of their input shape
#via the argument input_dim, and some 3D temporal layers support the arguments input_dim and input_length.

#Embedding is for to compress the input feature space into a smaller one
#Labmda is for 
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda


C:\Users\Rezwana\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
cbow=Sequential()
cbow.add(Embedding(input_dim=1089, output_dim=50, input_length=4))
cbow.add(Lambda(lambda x:K.mean(x, axis=1), output_shape=(50,))) 
cbow.add(Dense(1089, activation='softmax')) #softmax is used for multiclass classifier 

In [ ]:

cbow.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta')  


In [ ]:
train_x=inp+1
train_y=target_positive+1

In [ ]:
train_x.shape

In [ ]:
np.max(train_y)

In [ ]:
cbow.fit(train_x, train_y, batch_size=128, epochs=10)

In [ ]:
cbow.predict_classes(train_x[180].reshape((1, 4)))

In [ ]:
train_y[180]